In [ ]:
import glob
import os
import numpy as np
import cv2
def padding_square_resize_evi(im):
  img = im.transpose([1,2,0])
  old_image_height, old_image_width, channels = img.shape
  new_image_width = max(img.shape)
  new_image_height = max(img.shape)
  color = (0)
  result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

  x_center = (new_image_width - old_image_width) // 2
  y_center = (new_image_height - old_image_height) // 2
  
  result[y_center:y_center+old_image_height, 
          x_center:x_center+old_image_width] = img
  return cv2.resize(result,(32,32))
def onehot(label):
  a = np.zeros(4)
  a[int(label)-1]=1
  return a

In [ ]:
import pandas as pd
df = pd.read_csv('/content/labels.csv', header = None)
df

In [ ]:
import rasterio
import cv2
nocloud = '/content/drive/MyDrive/arv/Nocloud/'
data_all= []
labels_all = []
labels = []
for foldername in sorted(os.listdir(nocloud)):
  bound = []
  for filename in sorted([os.path.basename(a) for a in glob.glob(nocloud + foldername + '/*.tif')]):
    with rasterio.open(nocloud + foldername + '/' + filename) as src:
        band = src.read()
    bound.append(padding_square_resize_evi(band))
  data_all.append(np.array(bound))
  labels_all.append(onehot(df.iloc[int(filename)-1][1]))
  labels.append(df.iloc[int(filename)-1][1])

In [ ]:
data_all = np.array(data_all)
labels_all = np.array(labels_all)
print(np.array(data_all).shape,np.array(labels_all).shape)

In [ ]:
da = np.transpose(data_all, [0,2,3,1])
da =da.reshape(1317,32,38*32,-1)
da.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(da , labels_all, test_size=0.2 , shuffle = True)
print(x_train.shape, y_train.shape)

# Model

In [ ]:
import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers
from tensorflow.keras.layers import MaxPool2D, Conv2D, Dropout, Dense,Flatten

inputs = layers.Input(shape=(x_train.shape[1] , x_train.shape[2], x_train.shape[3]))
x = Conv2D(64, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu')(inputs)
x = Conv2D(32, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Dropout(0.25)(x)

x = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(4, activation='softmax')(x)
model = tf.keras.Model(inputs, x)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = "balanced" , classes = np.unique(labels) , y = np.array(labels))
class_weights = {i:w for i,w in enumerate(class_weights)}
class_weights

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

modelPath = '/content/model{epoch:02d}.hdf5'
earlystopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001, 
    patience=8, 
    restore_best_weights=True, 
    )
callbacksList = [earlystopping] #, callbacks=callbacksList
model_history = model.fit( x_train , y_train, epochs=100,  callbacks=callbacksList, batch_size = 64, validation_data = ( x_test, y_test ) , class_weight = class_weights)

# Submit